In [1]:
import findspark
findspark.init('/opt/spark')  # Adjust the path if Spark is installed elsewhere

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("example").getOrCreate()

24/07/05 22:13:52 WARN Utils: Your hostname, chuot-HP-Pavilion-Laptop-14-ce3xxx resolves to a loopback address: 127.0.1.1; using 192.168.1.14 instead (on interface wlo1)
24/07/05 22:13:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/05 22:13:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/05 22:13:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# Define the path to your JSON file
from pyspark.sql.functions import col, udf, to_date, when, regexp_extract, explode, col, regexp_replace, to_timestamp, concat, lit, trim, date_format
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType, ArrayType
import json
import re
import pymongo
json_file_path = 'careerviet.json'
# Read the JSON file into a DataFrame
df = spark.read.option("multiline","true").json(json_file_path)


# df_exploded = df.select(explode(col("category")).alias("unique_category"))

# unique_categories = df_exploded.select("unique_category").distinct().collect()
# print(len(unique_categories))
# for row in unique_categories:
#     pass
#     print(row.unique_category)
# unique_experience_df = df.select("location").distinct()

# unique_experience_df.show()
# from pyspark.sql.functions import col
unique_values = df.select(col("sex")).distinct().collect()
# unique_values.show()
# In ra các giá trị duy nhất
for row in unique_values:
    if row.sex is None:
        print("None ne")
    print(row.sex)


# print(df.size)


#rdd.toDF().select("salary").show(1)

# Salary


Nữ
Nam
None ne
None


In [40]:
df = spark.read.option("multiline","true").json(json_file_path)
print(df.count())
df = df.dropDuplicates(["title"])
print(df.count())
# df.write.json("careervietclean.json", mode='overwrite')
df1, df2, df3 = df.randomSplit([0.4, 0.3, 0.3], seed=42)
print(f"Count of the first half: {df1.count()}")
print(f"Count of the second half: {df2.count()}")
print(f"Count of the third half: {df3.count()}")

11521
8531


Count of the first half: 3457


Count of the second half: 2601


Count of the third half: 2473


In [47]:
df = df3

df = df.filter(col("company").isNotNull())

df = df.withColumn(
    "certificate",
     when(col("certificate") == None, 0)
    .when(col("certificate") == "Chứng chỉ", 1)
    .when(col("certificate") == "Trung học", 2)
    .when(col("certificate") == "Trung cấp", 3)
    .when(col("certificate") == "Cao đẳng", 4)
    .when(col("certificate") == "Đại học", 5)
    .when(col("certificate") == "Sau đại học", 6)
    .otherwise(0)
)

df = df.withColumn(
    "sex",
     when(col("sex") == "Nam", 1)
    .when(col("sex") == "Nữ", 2)
    .otherwise(0)
)

# Salary
def parse_salary(salary):
    vnd_per_usd = 25
    if salary is None:
        return {"type": 0 }
    elif (salary.strip().lower() == "cạnh tranh"):
        return {"type": 3 }
    elif "VND" in salary:
            salary = salary.replace(" VND", "").replace(" Tr", "").replace(",", ".").strip()
            if "-" in salary:
                min_salary, max_salary = salary.split("-")
                return {"type": 1, "min": float(min_salary), "max": float(max_salary) }
            elif "Trên" in salary:
                min_salary = salary.replace("Trên", "").strip()
                return {"type": 5, "min": float(min_salary)}
            elif "Lên đến" in salary:
                max_salary = salary.replace("Lên đến", "").strip()
                return {"type": 4, "max": float(max_salary)}
    elif "USD" in salary:
        salary = salary.replace(" USD", "").replace(" Tr", "").replace(",", "").strip()
        if "-" in salary:
            min_salary, max_salary = salary.split("-")
            return {"type": 1, "min": float(min_salary)* vnd_per_usd / 1000, "max": float(max_salary)* vnd_per_usd /1000}
        elif "Trên" in salary:
            min_salary = salary.replace("Trên", "").strip()
            return {"type": 5, "min": float(min_salary)* vnd_per_usd / 1000}
        elif "Lên đến" in salary:
            max_salary = salary.replace("Lên đến", "").strip()
            return {"type": 4, "min": float(max_salary)* vnd_per_usd / 1000}
    else:
        return {"type": 0 }
                
# Age
def parse_age(age):
    if age is None or age == "Không giới hạn tuổi":
        return { "type": 0 }
    # age = age.strip().lower()
    if "-" in age:
        strip_age = age.strip()
        min_age, max_age = age.strip().split("-")
        return { "type": 1, "min": int(min_age), "max": int(max_age) }
    elif "Trên" in age:
        min_age = age.replace("Trên ", "")
        return { "type": 4, "min": int(min_age)}
    elif "Dưới" in age:
        max_age = age.replace("Dưới ", "")
        return { "type": 3, "min": int(max_age)}  
    else:
        fixed = age.strip()
        return { "type": 2, "fixed": int(fixed)}  
    return { "type": 0 }

# Experience
def parse_experience(experience):
    if experience is None or experience == "Chưa có kinh nghiệm":
        return { "type": 0 }
    if "-" in experience:
        strip_experience = experience.replace("Năm", "").strip()
        min_exp, max_exp = strip_experience.split("-")
        return {"type": 1, "min": int(min_exp), "max": int(max_exp) }
    elif "Lên đến" in experience:
        strip_experience = experience.replace("Lên đến", "").replace("Năm", "").strip()
        max = strip_experience
        return { "type": 3, "max": int(max) }
    elif "Trên" in experience:
        strip_experience = experience.replace("Trên", "").replace("Năm", "").strip()
        min = strip_experience
        return { "type": 4, "min": int(min) }
    else:
        fixed = experience.replace("Năm", "").strip()
        return { "type": 2, "fixed": int(fixed)}
    return { "type": 0 }

# Expiration
df = df.withColumn("expiration", date_format(to_timestamp(concat(col("expiration"), lit(" 00:00:00")), "dd/MM/yyyy HH:mm:ss"), "yyyy-MM-dd HH:mm:ss"))

# Update at
df = df.withColumn("update_time", date_format(to_timestamp(concat(col("update_time"), lit(" 00:00:00")), "dd/MM/yyyy HH:mm:ss"), "yyyy-MM-dd HH:mm:ss"))

# Type
def get_type_job(type):
    if type == "Nhân viên chính thức":
        return 0
    elif type == "Bán thời gian":
        return 2
    elif type == "Thời vụ/ Nghề tự do":
        return 1
    elif type == "Thực tập":
        return 4
    else:
        return 5
def get_list_type_job(types):
    # Thời vụ/ Nghề tự do, Thực tập
    # type = types.split(",")
    if types is None:
        return []
    return [get_type_job(word.strip()) for word in types.split(",")]


classify_type_udf = udf(get_list_type_job, ArrayType(IntegerType()))
df = df.withColumn("type", classify_type_udf(col("type")))

# Role
def classify_role(role):
    if role is None:
        return 0
    role = role.strip()
    if role == "Nhân viên":
        return 0
    elif role == "Quản lý" or role == "Quản lý cấp trung":
        return 1
    elif role == "Cộng tác viên":
        return 6
    elif role == "Trưởng nhóm / Giám sát":
        return 5
    elif role == "Sinh viên/ Thực tập sinh" or role == "Mới tốt nghiệp":
        return 4
    elif role == "Giám đốc" or role == "Tổng giám đốc":
        return 2
    elif role == "Phó Giám đốc":
        return 3
    elif role == "Chuyên gia":
        return 7
    else:
        return 0
classify_role_udf = udf(classify_role, IntegerType())
df = df.withColumn("role", classify_role_udf(col("role")))

def normalize_benefit(benefits):
    if benefits is None:
        return []
    return list(map(lambda benefit: re.sub(r'^[*\-+]+', '', benefit).strip(), benefits))
benefit_udf = udf(normalize_benefit, ArrayType(StringType()))
df = df.withColumn("benefit", benefit_udf(col("benefit")))
df = df.withColumn("requirement", benefit_udf(col("requirement")))
df = df.withColumn("description", benefit_udf(col("requirement")))

def getProvince(province):
    if province == "An Giang": 
      location = "AG"
      
    elif province == "Bà Rịa - Vũng Tàu": 
      location = "BV"
      
    elif province == "Bạc Liêu": 
      location = "BL"
      
    elif province == "Bắc Kạn" or province == 'Bắc Cạn': 
      location = "BK"
      
    elif province == "Bắc Giang": 
      location = "BG"
      
    elif province == "Bắc Ninh": 
      location = "BN"
      
    elif province == "Bến Tre": 
      location = "BT"
      
    elif province == "Bình Dương": 
      location = "BD"
      
    elif province == "Bình Định": 
      location = "BDI"
      
    elif province == "Bình Phước": 
      location = "BP"
      
    elif province == "Bình Thuận": 
      location = "BT"
      
    elif province == "Cà Mau": 
      location = "CM"
      
    elif province == "Cao Bằng": 
      location = "CB"
      
    elif province == "Cần Thơ": 
      location = "CT"
      
    elif province == "Đà Nẵng": 
      location = "DN"
      
    elif province == "Đắk Lắk" or province == 'Dak Lak': 
      location = "DL"
      
    elif province == "Đắk Nông" or province == 'Dak Nông': 
      location = "DNO"
      
    elif province == "Điện Biên": 
      location = "DB"
      
    elif province == "Đồng Nai": 
      location = "DNA"
      
    elif province == "Đồng Tháp": 
      location = "DT"
      
    elif province == "Gia Lai": 
      location = "GL"
      
    elif province == "Hà Giang": 
      location = "HG"
      
    elif province == "Hà Nam": 
      location = "HNA"
      
    elif province == "Hà Nội": 
      location = "HN"
      
    elif province == "Hà Tĩnh": 
      location = "HT"
      
    elif province == "Hải Dương": 
      location = "HD"
      
    elif province == "Hải Phòng": 
      location = "HP"
      
    elif province == "Hậu Giang": 
      location = "HGI"
      
    elif province == "Hòa Bình": 
      location = "HB"

    elif province == 'Hồ Chí Minh' or province == "TP.HCM" : 
      location = "HCM"
      
    elif province == "Hưng Yên": 
      location = "HY"
      
    elif province == "Khánh Hòa": 
      location = "KH"
      
    elif province == "Kiên Giang": 
      location = "KG"
      
    elif province == "Kon Tum": 
      location = "KT"
      
    elif province == "Lai Châu": 
      location = "LC"
      
    elif province == "Lạng Sơn": 
      location = "LS"
      
    elif province == "Lào Cai": 
      location = "LCA"
      
    elif province == "Lâm Đồng": 
      location = "LD"
      
    elif province == "Long An": 
      location = "LA"
      
    elif province == "Nam Định": 
      location = "ND"
      
    elif province == "Nghệ An": 
      location = "NA"
      
    elif province == "Ninh Bình": 
      location = "NB"
      
    elif province == "Ninh Thuận": 
      location = "NT"
      
    elif province == "Phú Thọ": 
      location = "PT"
      
    elif province == "Phú Yên": 
      location = "PY"
      
    elif province == "Quảng Bình": 
      location = "QB"
      
    elif province == "Quảng Nam": 
      location = "QNA"
      
    elif province == "Quảng Ngãi": 
      location = "QNG"
      
    elif province == "Quảng Ninh": 
      location = "QN"
      
    elif province == "Quảng Trị": 
      location = "QT"
      
    elif province == "Sóc Trăng": 
      location = "ST"
      
    elif province == "Sơn La": 
      location = "SL"
      
    elif province == "Tây Ninh": 
      location = "TN"
      
    elif province == "Thái Bình": 
      location = "TB"
      
    elif province == "Thái Nguyên": 
      location = "TNG"
      
    elif province == "Thanh Hóa": 
      location = "TH"
      
    elif province == "Thừa Thiên- Huế" or province == "Thừa Thiên Huế": 
      location = "TTH"
      
    elif province == "Tiền Giang": 
      location = "TG"
      
    elif province == "Trà Vinh": 
      location = "TV"
      
    elif province == "Tuyên Quang": 
      location = "TQ"
      
    elif province == "Vĩnh Long": 
      location = "VL"
      
    elif province == "Vĩnh Phúc": 
      location = "VP"
      
    elif province == "Yên Bái": 
      location = "YB"
    else:
        print("No match:\""+ province+ "\"")
        location = 'other'
    return location

# location
def normalize_location(provinces, locations):
    arr = []
    if provinces is None:
        provinces = []
    if locations is None:
        locations = []
    if len(locations) == 0 and len(provinces) == 0:
        return []
    if len(locations) == 0 and len(provinces) > 0:
        for province in provinces:
            arr.append({"province": getProvince(province), "district": None, "address": province})
    else:   
        for province, location in zip(provinces, locations):
            parts = [part.strip() for part in location.split(',') if part.strip()]
            if len(parts) >= 2:
                job_province = parts[-1]
                district = None
                if job_province == province:
                    district = parts[-2]
                else:
                    district = [address for address in parts if 'huyện' in address.lower() or 'Q.' in address or 'Quận' in address.lower()]
                arr.append({"province": getProvince(province), "district": district, "address": location})
            else: 
                arr.append({"address": location, "district": None, "province": getProvince(province)})
    return arr

category_mapping = {
    "Du lịch": "dv",
    "Thủy lợi": "nn_ln_ts",
    "Công nghệ thực phẩm / Dinh dưỡng": "tp",
    "Lâm Nghiệp": "nn_ln_ts",
    "In ấn / Xuất bản": "xb",
    "An toàn lao động": "at_an",
    "Hành chính / Thư ký": "hc_ql",
    "CNTT - Phần mềm": "it",
    "Quảng cáo / Đối ngoại / Truyền Thông": "tt_mkt",
    "Nông nghiệp": "nn_ln_ts",
    "Ngân hàng": "tc_kt",
    "Quản lý điều hành": "hc_ql",
    "Kiến trúc": "tk_kt_nt",
    "Mỹ thuật / Nghệ thuật / Thiết kế": "tk_kt_nt",
    "Trắc địa / Địa Chất": "kh_kt",
    "Điện / Điện tử / Điện lạnh": "kh_kt",
    "Hàng gia dụng / Chăm sóc cá nhân": "kd",
    "Chứng khoán": "tc_kt",
    "Bảo trì / Sửa chữa": "cn_sx",
    "Tổ chức sự kiện": "dv",
    "Sản xuất / Vận hành sản xuất": "cn_sx",
    "Nhà hàng / Khách sạn": "dv",
    "Hàng không": "vt",
    "Luật / Pháp lý": "law",
    "Bất động sản": "bds",
    "Tư vấn": "dv",
    "Dịch vụ khách hàng": "dv",
    "Y tế / Chăm sóc sức khỏe": "yt_sk",
    "Công nghệ sinh học": "kh_kt",
    "Giáo dục / Đào tạo": "gd_dt",
    "Chăn nuôi / Thú y": "nn_ln_ts",
    "An Ninh / Bảo Vệ": "at_an",
    "CNTT - Phần cứng / Mạng": "it",
    "Dầu khí": "cn_sx",
    "Môi trường": "cn_sx",
    "Bán lẻ / Bán sỉ": "kd",
    "Tài chính / Đầu tư": "tc_kt",
    "Thủy sản / Hải sản": "nn_ln_ts",
    "Phi chính phủ / Phi lợi nhuận": "dv",
    "Kế toán / Kiểm toán": "tc_kt",
    "Bán hàng / Kinh doanh": "kd",
    "Xuất nhập khẩu": "vt",
    "Giải trí": "dv",
    "Quản lý chất lượng (QA/QC)": "hc_ql",
    "Đồ gỗ": "cn_sx",
    "Hóa học": "kh_kt",
    "Nội ngoại thất": "tk_kt_nt",
    "Nhân sự": "hc_ql",
    "Hàng hải": "vt",
    "Dược phẩm": "yt_sk",
    "Vận chuyển / Giao nhận /Kho vận": "vt",
    "Thống kê": "pt_tk",
    "Tiếp thị / Marketing": "tt_mkt",
    "Cơ khí / Ô tô / Tự động hóa": "kh_kt",
    "Xây dựng": "xd",
    "Biên phiên dịch": "dv",
    "Bảo hiểm": "law",
    "Truyền hình / Báo chí / Biên tập": "tt_mkt",
    "Ngành khác": "other",
    "Bưu chính viễn thông": "tt_mkt",
    "Thu mua / Vật tư": "kd",
    "Dệt may / Da giày / Thời trang": "cn_sx",
    "Mới tốt nghiệp / Thực tập": "other",
    "Lao động phổ thông": "ldpt",
    "Thực phẩm & Đồ uống": "tp",
    "Tiếp thị trực tuyến": "tt_mkt",
    "Khác": "other",
    "Thư viện": "hc_ql",
    "Khoáng sản": "cn_sx"
}

def parse_category(categories):
    if categories is None or len(categories) == 0:
        return []
    else :
        category_arr = []
        for category in categories:
            if category in category_mapping:
                category_arr.append(category_mapping[category])
            else:
                print("No result field: "+ category)
                category_arr.append("other")
        return category_arr

# df.select("title").show(5)
rdd = df.rdd.map(lambda row: {
    **row.asDict(),
    "salary": parse_salary(row["salary"]),
    "age": parse_age(row["age"]),
    "location": normalize_location(row["province"], row["location"]),
    "experience": parse_experience(row["experience"]),
    "field": parse_category(row["category"])
})


# rdd.count()

# df.select(["role", "url"]).show(30, False)
# df.select("location").show(1, False)

# rdd.collect()

# Insert dữ liệu vào collection "jobs"
def insert_into_mongodb(partition):
    client = pymongo.MongoClient("mongodb://localhost:27017/", username='admin', password='20194856')
    db = client["thesis"]
    collection = db["jobs"]

    for record in partition:
        try:
            collection.insert_one(record)
        except Exception as e:
            print("ERROR when insert"+ str(e))

# Insert dữ liệu vào collection "jobs"
rdd.foreachPartition(insert_into_mongodb)
print("Insert sucess to mongodb: "+ str(rdd.count())+ " jobs")

No match:"Quốc tế"
No match:"Đồng Bằng Sông Cửu Long"
No match:"KV Tây Nguyên"
No match:"KV Đông Nam Bộ"
No match:"Quốc tế"
No match:"Khác"
No match:"KV Đông Nam Bộ"
No match:"Quốc tế"
No match:"Malaysia"                                                 (0 + 1) / 1]
No match:"Khác"
No match:"Đồng Bằng Sông Cửu Long"
24/07/05 23:59:39 ERROR Executor: Exception in task 0.0 in stage 452.0 (TID 261)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 452.0 failed 1 times, most recent failure: Lost task 0.0 in stage 452.0 (TID 261) (192.168.1.14 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/spark/python/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  [Previous line repeated 1 m